<a href="https://colab.research.google.com/github/naguzmans/opportunistic-atm/blob/master/00_Download_OpenSky_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Dependencies

In [ ]:
!pip install pyopensky --upgrade
!pip install haversine

try:
  from pyopensky import OpenskyImpalaWrapper
except:
  pass

!cp /content/drive/MyDrive/opportunistic-utm/config/secret.conf /root/.config/pyopensky/

# Run Query and Download Data

In [ ]:
from pyopensky import OpenskyImpalaWrapper
import pandas as pd
import datetime
from haversine import haversine, inverse_haversine, Unit
import numpy as np
from pyopensky import OpenskyImpalaWrapper
import time
from tqdm import tqdm
from IPython.display import clear_output
import os

# Time of query
def timestamp_to_epoch(timestamp):
  unix_epoch = datetime.datetime(1970, 1, 1)
  log_dt = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
  seconds_from_epoch = (log_dt - unix_epoch).total_seconds()
  return seconds_from_epoch

year = '2015'
start_month = 1
end_month = 12

folder_path = '/content/drive/MyDrive/opportunistic-utm'

for i in range(start_month, end_month + 1):
  month = str(i).zfill(2)
  for day in tqdm(range(1,32)):
    try:
      from_time_str = f'{year}-{month}-{str(day).zfill(2)} 00:00:00'
      to_time_str = f'{year}-{month}-{str(day).zfill(2)} 23:59:59'
      from_time = timestamp_to_epoch(from_time_str)
      to_time = timestamp_to_epoch(to_time_str)

      # Area of query
      airport = (51.871496514, -0.367665196) # Luton
      # airport = (51.470020, -0.454295) # Heathrow
      diag_distance = 25
      min_coord = inverse_haversine(airport, diag_distance, np.pi * 1.25)
      max_coord = inverse_haversine(airport, diag_distance, np.pi * .25)
      min_lat, max_lat, min_lon, max_lon = (min_coord[0], max_coord[0], min_coord[1], max_coord[1])

      # Height of query
      min_height, max_height = (0, 1000)

      # OpenSky wrapper
      start = time.time()
      print(f'\nMonth: {month} / Day: {day}')
      opensky = OpenskyImpalaWrapper()

      opensky_query = opensky.rawquery(
          f'SELECT * FROM state_vectors_data4 \
          WHERE lat >= {min_lat} AND lat <= {max_lat} \
          AND lon >= {min_lon} AND lon <= {max_lon} \
          AND hour >= {from_time} AND hour <= {to_time} \
          AND geoaltitude >= {min_height} AND geoaltitude <= {max_height};'
      )

      end = time.time()
      print(f'Time elapsed: {np.round(end - start,1)}s')
      print(f'Time elapsed: {np.round((end - start)/60,1)}m')

      unique_callsigns = pd.unique(opensky_query['callsign'])
      print(f'Query length: {len(unique_callsigns)}')

      # Save CSV
      if not os.path.exists(f'{folder_path}/data/{year}/{year}-{month}/'):
        os.makedirs(f'{folder_path}/data/{year}/{year}-{month}/')

      opensky_query.to_csv(f'{folder_path}/data/{year}/{year}-{month}/{year}-{month}-{str(day).zfill(2)}.csv', index=False)
      clear_output(wait=False)
    except:
      break
      clear_output(wait=False)